In [52]:
from langchain_core.tools import tool, InjectedToolArg
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from typing import Annotated
from dotenv import load_dotenv
import requests
load_dotenv()

True

In [53]:
@tool
def get_CurrencyCon_Tool(base_Currency: str, target_Currency: str):
    """The function fetches the currency conversion factor between a given base currency and a target currency"""
    #url = " 3103c3c22f2f8a819d33470b"
    url = f'https://v6.exchangerate-api.com/v6/3103c3c22f2f8a819d33470b/pair/{base_Currency}/{target_Currency}'
    response = requests.get(url)
    return response.json()
@tool 
def convert(base_Currency:float, conversion_rate: Annotated[float, InjectedToolArg])->float:
    """Given a currency conversion rate this function calculates the target currency from a given base currency value"""
    return base_Currency*conversion_rate


In [54]:
get_CurrencyCon_Tool.invoke({'base_Currency':"USD", "target_Currency": "PKR"})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749513602,
 'time_last_update_utc': 'Tue, 10 Jun 2025 00:00:02 +0000',
 'time_next_update_unix': 1749600002,
 'time_next_update_utc': 'Wed, 11 Jun 2025 00:00:02 +0000',
 'base_code': 'USD',
 'target_code': 'PKR',
 'conversion_rate': 282.2975}

In [55]:
convert.invoke({"base_Currency": 12.2, "conversion_rate": 2.1})

25.62

In [56]:
# tool Bending
llm = ChatOpenAI()
llmWith_Tools = llm.bind_tools([get_CurrencyCon_Tool, convert])

In [57]:
message = [HumanMessage("What is the conversion factor between USD and PKR, and Based on that can you convert 10 to PKR")]

In [58]:
ai_message = llmWith_Tools.invoke(message)
message.append(ai_message)

In [59]:
ai_message.tool_calls

[{'name': 'get_CurrencyCon_Tool',
  'args': {'base_Currency': 'USD', 'target_Currency': 'PKR'},
  'id': 'call_MUWd9rn7qilAMcSNTyKfcvyR',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_Currency': 10},
  'id': 'call_jv3uphgv7Ztadw2HSXt1WcZk',
  'type': 'tool_call'}]

In [60]:
import json
for tool_call in ai_message.tool_calls:
    # Execute the first tool
    if tool_call["name"] == "get_CurrencyCon_Tool":
        tool_message = get_CurrencyCon_Tool.invoke(tool_call)
        conversion_rate = json.loads(tool_message.content)["conversion_rate"]
        message.append(tool_message)   
    if tool_call["name"] == "convert":
        tool_call['args']['conversion_rate'] = conversion_rate
        toolCall_message = convert.invoke(tool_call)   
        message.append(toolCall_message)      
        

In [63]:
llmWith_Tools.invoke(message).content

'The conversion factor between USD and PKR is 282.2975. \nTherefore, 10 USD is equivalent to 2822.98 PKR.'